<a href="https://colab.research.google.com/github/unclepete-20/lab3-sds/blob/main/lab3_sds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laboratorio de Detección de Malware mediante Secuencia de Llamadas a las APIs**

## Integrantes:
- Pedro Arriola (20188)
- Andres de la Roca (20332)

## Introducción

En este laboratorio, exploraremos técnicas de detección de malware utilizando el análisis dinámico de la secuencia de llamadas a las APIs. El análisis dinámico ofrece información valiosa sobre el comportamiento de un malware y cómo interactúa con el sistema infectado. En lugar de depender únicamente del análisis estático, que puede ser fácilmente obstruido por técnicas de ofuscación, el análisis dinámico nos permite observar el orden en el tiempo en el que se ejecutan las llamadas a las APIs, lo que puede revelar patrones característicos de malware.

En este laboratorio, utilizaremos un conjunto de datos que contiene la secuencia de llamadas a las APIs generadas por muestras de malware y software benigno. Implementaremos varios modelos de aprendizaje automático, incluidos Logistic Regression, Support Vector Machine (SVM) y Random Forest, para detectar y distinguir entre malware y software benigno basándonos en estas secuencias de llamadas a las APIs.

A lo largo de este laboratorio, cubriremos las siguientes etapas del proceso de aprendizaje automático: exploración de datos, preprocesamiento de datos, ingeniería de características, implementación de modelos y evaluación de modelos utilizando métricas como precisión, recall y ROC AUC. Al finalizar el laboratorio, seremos capaces de comparar y evaluar la efectividad de diferentes modelos en la detección de malware basada en la secuencia de llamadas a las APIs.


In [1]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv("MalBehavD-V1-dataset.csv")

# Visualizar las primeras filas del dataset
print(df.head())

# Información del dataset
print(df.info())

                                              sha256  labels  \
0  5c18291c481a192ed5003084dab2d8a117fd3736359218...       0   
1  4683faf3da550ffb594cf5513c4cbb34f64df85f27fd1c...       0   
2  9a0aea1c7290031d7c3429d0e921f107282cc6eab854ee...       0   
3  e0f3e4d5f50afd9c31e51dd9941c5a52d57c7c524f5d11...       0   
4  ec2b6d29992f13e74015ff0b129150b4afae15c593e4b7...       0   

                     0                        1                   2  \
0         LdrUnloadDll           CoUninitialize          NtQueryKey   
1         NtOpenMutant      GetForegroundWindow          NtQueryKey   
2  GetForegroundWindow              DrawTextExW       GetSystemInfo   
3      NtQueryValueKey             LdrUnloadDll  GlobalMemoryStatus   
4         LdrUnloadDll  GetSystemTimeAsFileTime           NtOpenKey   

                   3                            4                       5  \
0  NtDuplicateObject            GetShortPathNameW           GetSystemInfo   
1        DrawTextExW         NtSet

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
# Preprocesamiento de texto

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Eliminar caracteres especiales
    text = re.sub(r'\d+', ' ', text)  # Eliminar números
    text = text.lower()  # Convertir a minúsculas
    tokens = word_tokenize(text)  # Tokenización
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]  # Stemming y eliminación de stopwords
    return ' '.join(tokens)

# Concatenar las secuencias de llamadas a las APIs en una sola cadena de texto por fila
df['api_sequence_text'] = df.drop(['sha256', 'labels'], axis=1).apply(lambda x: ' '.join(x.astype(str)), axis=1)

# Preprocesamiento de texto
df['api_sequence_text_processed'] = df['api_sequence_text'].apply(preprocess_text)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorización del texto usando TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Especificar el número máximo de características
X_tfidf = tfidf_vectorizer.fit_transform(df['api_sequence_text_processed'])

# Dividir los datos en características (X) y etiquetas (y)
X = X_tfidf
y = df['labels']

In [16]:
from sklearn.model_selection import train_test_split, cross_validate

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import cross_val_score

# 4. Implementación y Validación de Modelos
logistic_model = LogisticRegression()
svm_model = SVC()
rf_model = RandomForestClassifier()

models = [logistic_model, svm_model, rf_model]

for model in models:
    scores = cross_validate(model, X_train, y_train, cv=10, scoring=('accuracy', 'precision', 'recall', 'roc_auc'))
    print("Model:", model.__class__.__name__)
    print("Accuracy:", scores['test_accuracy'].mean())
    print("Precision:", scores['test_precision'].mean())
    print("Recall:", scores['test_recall'].mean())
    print("ROC AUC:", scores['test_roc_auc'].mean())

# 5. Evaluación del Modelo en el Conjunto de Prueba
best_model = max(models, key=lambda model: cross_val_score(model, X_train, y_train, cv=10).mean())
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print("Best Model:", best_model.__class__.__name__)
print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))
print("Precision on Test Set:", precision_score(y_test, y_pred))
print("Recall on Test Set:", recall_score(y_test, y_pred))
print("ROC AUC on Test Set:", roc_auc_score(y_test, y_pred))

Model: LogisticRegression
Accuracy: 0.9321942892613283
Precision: 0.9591172795675522
Recall: 0.9027465667915106
ROC AUC: 0.9600575387298234
Model: SVC
Accuracy: 0.9344165114835505
Precision: 0.9582263450359709
Recall: 0.9082896379525593
ROC AUC: 0.9694432135349738
Model: RandomForestClassifier
Accuracy: 0.9572004965859714
Precision: 0.9812992510363436
Recall: 0.9317478152309613
ROC AUC: 0.9890591179523763
Best Model: RandomForestClassifier
Accuracy on Test Set: 0.9610894941634242
Precision on Test Set: 0.9865229110512129
Recall on Test Set: 0.9360613810741688
ROC AUC on Test Set: 0.9614517431686633


## **Discusion de resultados**

| Modelo                    | Accuracy | Precision | Recall  | ROC AUC |
|---------------------------|----------|-----------|---------|---------|
| Logistic Regression       | 0.9322   | 0.9591    | 0.9027  | 0.9601  |
| Support Vector Machine    | 0.9344   | 0.9582    | 0.9083  | 0.9694  |
| Random Forest             | 0.9572   | 0.9813    | 0.9317  | 0.9891  |


En base a los resultados obtenidos, podemos observar que el modelo Random Forest supera a los otros dos modelos (Logistic Regression y Support Vector Machine) en todas las métricas evaluadas (Accuracy, Precision, Recall y ROC AUC) tanto en el conjunto de entrenamiento como en el conjunto de prueba.

La precisión de un modelo es importante en el contexto de la detección de malware, ya que queremos minimizar los falsos positivos, es decir, identificar correctamente el malware sin clasificar incorrectamente software benigno como malicioso. El modelo Random Forest muestra una precisión superior en comparación con los otros modelos, lo que indica que es más efectivo para identificar correctamente el malware y reducir los falsos positivos.

Además, el modelo Random Forest también muestra una mayor recall en comparación con los otros modelos. Un alto recall significa que el modelo es capaz de identificar la mayoría de las instancias de malware en el conjunto de datos, lo que es crucial en la detección de amenazas en seguridad.

La combinación de alta precisión y alto recall, junto con un ROC AUC más alto, indica que el modelo Random Forest es capaz de encontrar un equilibrio óptimo entre la capacidad de identificar el malware y minimizar los falsos positivos, lo que lo convierte en el mejor modelo para este caso de detección de malware basado en la secuencia de llamadas a las APIs en este conjunto de datos específico.